# Silver Layer Processing
Simple wrapper to process Bronze data through Silver transformations

In [ ]:
# Install updated wheels with YAML configs
%pip install /lakehouse/default/Files/unified_etl_core-1.0.0-py3-none-any.whl
%pip install /lakehouse/default/Files/unified_etl_connectwise-1.0.0-py3-none-any.whl

In [ ]:
# Process all entities through Silver
# Map config entity names to actual table names
entity_mapping = {
    "agreement": "agreement",
    "time_entry": "timeentry",
    "expense_entry": "expenseentry",
    "product_item": "productitem",
    "invoice": "postedinvoice",  # Note: this was the 0-record table, may want 'invoice' instead
}

lakehouse_root = "/lakehouse/default/Tables/"

for config_entity, table_entity in entity_mapping.items():
    bronze_path = f"{lakehouse_root}bronze/bronze_cw_{table_entity}"
    print(f"\nProcessing {config_entity} (from {bronze_path}) through Silver...")
    try:
        # Check if Bronze table exists first
        df = spark.read.format("delta").load(bronze_path)
        count = df.count()
        print(f"  Found {count} records in Bronze")

        if count > 0:
            run_bronze_to_silver_pipeline(
                entity_name=config_entity,
                bronze_table_name=f"bronze_cw_{table_entity}",
                lakehouse_root=lakehouse_root,
            )
            print(f"✅ {config_entity} Silver processing complete")
        else:
            print(f"⚠️  Skipping {config_entity} - no data in Bronze")
    except Exception as e:
        print(f"❌ {config_entity} failed: {e!s}")

In [ ]:
# Verify Silver tables
for config_entity, table_entity in entity_mapping.items():
    silver_path = f"{lakehouse_root}silver/silver_cw_{config_entity}"
    try:
        df = spark.read.format("delta").load(silver_path)
        count = df.count()
        print(f"{config_entity}: {count} records in Silver")
    except Exception as e:
        print(f"{config_entity}: Error reading Silver table - {e!s}")

In [ ]:
# Verify Silver tables
for entity in entities:
    silver_path = f"{lakehouse_root}silver/silver_cw_{entity}"
    try:
        df = spark.read.format("delta").load(silver_path)
        count = df.count()
        print(f"{entity}: {count} records in Silver")
    except Exception as e:
        print(f"{entity}: Error reading Silver table - {e!s}")